In [0]:
import requests, json, time, urllib3, io
import matplotlib.pyplot as plt
import pandas as pd
import datetime as dt

In [0]:
def load(url, printout=False, remove_bottom_rows=0, remove_columns =[]):
  headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}
  response = requests.get(url, headers = headers)
  df = pd.read_json(response.text)
  
  if remove_bottom_rows > 0:
    df.drop(df.tail(remove_bottom_rows).index, inplace=True)
    
  df.drop(columns=remove_columns, axis=1, inplace=True)
  
  df = df.dropna(axis=1)
  
  if printout:
    print(df.head())
    print(df.shape)
    
  return df

  

In [0]:
def get_trades_for_product(product_id):
  type = '/trades'
  url = "https://api.gdax.com/products/" + product_id + type
  return load(url)

In [0]:
def initialize_blotter():
  col_names = ['Timestamp', 'Pair', 'Quantitiy', 'Executed Price']
  df = pd.DataFrame(columns = col_names)
  return df, col_names
  

In [0]:
def get_price(pair):
  type = '/book'
  url = "https://api.gdax.com/products/" + pair + type
  df = load(url)
  ask = df.iloc[0]['asks'][0]
  bid = df.iloc[0]['bids'][0]
  return bid, ask

In [0]:
def initialize_pl(pairs):
  col_names = ['Pair', 'Positions', 'VMAP', 'UPL', 'RPL']
  pl = pd.DataFrame(columns=col_names)
  for p in pairs:
    data = pd.DataFrame([[p,0,0,0,0]], columns = col_names)
    pl = pl.append(data, ignore_index = True)
  return pl, col_names

In [0]:
def trade(blotter, pl, qty, pair):
  bid, ask = get_price(pair)
  if qty > 0:
    price = ask
  else:
    price = bid
  
  data = pd.DataFrame([[dt.datetime.now(), pair, qty, price]], columns=['Timestamp', 'Pair', 'Quantitiy', 'Executed Price'])
  
  blotter = blotter.append(data, ignore_index = True)
  return blotter

In [21]:
pairs = ['eth-usd', 'btc-usd']
blotter, col_names = initialize_blotter()
pl, pl_col_names = initialize_pl(pairs)


#Coin Codes
ETH_USD = 0
BTC_USD = 1

blotter = trade(blotter, pl, 1, pairs[ETH_USD])
blotter = trade(blotter, pl, 2, pairs[ETH_USD])
blotter = trade(blotter, pl, -1, pairs[BTC_USD])

print(blotter)

                   Timestamp     Pair Quantitiy Executed Price
0 2018-05-29 20:04:17.848874  eth-usd         1         563.41
1 2018-05-29 20:04:17.985142  eth-usd         2         563.41
2 2018-05-29 20:04:18.133557  btc-usd        -1        7456.74
